In [19]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [20]:
sales_data = pd.read_csv("./data/sales.csv")
sales_data.head()

print(sales_data.state_holiday.unique())

le = LabelEncoder()
sales_data.state_holiday = le.fit_transform(sales_data.state_holiday)

['0' 'a' 'c' 'b']


In [22]:
sales_data.head()

print(sales_data.dtypes)

Unnamed: 0              int64
store_ID                int64
day_of_week             int64
date                   object
nb_customers_on_day     int64
open                    int64
promotion               int64
state_holiday           int64
school_holiday          int64
sales                   int64
dtype: object
